In [ ]:
# This file is for benchmarking the performance of Unet with different loss functions 

In [0]:
import numpy as np
import tensorflow as tf
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn import metrics
from keras.callbacks import ModelCheckpoint
from keras.utils import to_categorical
import os
import sys

In [0]:
from google.colab import drive
drive.mount('/content/gdrive')

In [0]:
train_images = np.load("/content/gdrive/My Drive/augmented20/images_train.npy")
train_labels = np.load("/content/gdrive/My Drive/augmented20/labels_train.npy")


In [0]:
num_samp=10000
imgs_np=train_images[0:num_samp,:,:]
imgs_np.shape

In [0]:
masks_np=train_labels[0:num_samp,:,:]
masks_np.shape

In [0]:
x = np.asarray(imgs_np, dtype=np.float32)/imgs_np.max()
y= masks_np

In [0]:
def make_mask(img):
    mask = np.zeros((img.shape[0], img.shape[1], 3))
    mask[:,:,0] = np.logical_or(mask[:,:,0],(img==0))
    mask[:,:,1] = np.logical_or(mask[:,:,1],(img==1))
    mask[:,:,2] = np.logical_or(mask[:,:,2],(img==2))
    #mask = np.resize(mask,(img.shape[0]*img.shape[1], 3))
    return mask

In [0]:
y_mask=np.zeros((y.shape[0], y.shape[1], y.shape[2], 3))
for i in range(0, y.shape[0]):
    y_mask[i,:,:,:]=make_mask(y[i,:,:])
print(x.shape, y_mask.shape)

In [0]:
x = x.reshape(x.shape[0], x.shape[1], x.shape[2], 1)
#y = y.reshape(y.shape[0], y.shape[1], y.shape[2], 1)
x.shape

In [0]:
yl=y_mask

In [0]:
from sklearn.model_selection import train_test_split

x_train, x_val, y_train, y_val = train_test_split(x, yl, test_size=0.2, random_state=0)

print("x_train: ", x_train.shape)
print("y_train: ", y_train.shape)
print("x_val: ", x_val.shape)
print("y_val: ", y_val.shape)

In [0]:
!pip install segmentation-models

In [0]:
from segmentation_models import Unet 
from segmentation_models import get_preprocessing
from segmentation_models.losses import bce_jaccard_loss, DiceLoss, CategoricalCELoss, CategoricalFocalLoss
from segmentation_models.metrics import iou_score

In [0]:
# define number of channels
N = x.shape[-1]

In [0]:
model = Unet(backbone_name='vgg16', encoder_weights=None, input_shape=(None, None, N), classes=3, activation ='softmax')

In [0]:
model.compile('Adam', loss=CategoricalCELoss(), metrics=[iou_score])
#model.compile('Adam', loss=CategoricalFocalLoss(), metrics=[iou_score])
#model.compile('Adam', loss=DiceLoss(), metrics=[iou_score])
#model.compile('Adam', loss=bce_jaccard_loss, metrics=[iou_score])

In [0]:
import tensorflow as tf 
tf.test.gpu_device_name() 

In [0]:
model.fit(
    x=x_train,
    y=y_train,
    epochs=10,
    validation_data=(x_val, y_val),
)

In [0]:
#  We reprot the average of metrics on validation data over epaochs and do the comparison of performance of losses